# 6.4: Resampling Exercises

## Getting Started

### Import Libraries 

We import our standard libraries and specific objects/libraries at the top level of our notebook.

In [55]:
# Load our previous libraries and objects
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import train_test_split
from ISLP.models import (ModelSpec as MS, summarize)

# Import new libraries and objects
from sklearn.model_selection import cross_validate
from ISLP.models import sklearn_sm

import warnings 
warnings.filterwarnings('ignore') # mute warning messages

## The Validation Set Approach

We explore the use of the validation set approach in order to estimate the test error rates that result from fitting various linear models on the `Auto` data set.

We use the function `train_test_split()` to split the data into training and validation sets. As there are 392 observations, we split into two equal sets of size 196 using the argument `test_size=196`. We want to set a random seed (`random_state=0`) so that the random results can be reproduced again.

In [56]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,
                                         test_size=0.50, # split in two - 50% for TRAIN and 50% for VALID/TEST
                                         random_state=0) # random seed

# sample size 392 records
# parameter: test_size 
#     - if INTEGER (test_size = 100)
#             - auto_train will have 292 records  
#             - auto_valid will have 100 records 
#     - if FLOAT (test_size = 0.2)
#             - auto_train will have 80% records  
#             - auto_valid will have 20% records 

In [57]:
print (Auto)


      mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
0    18.0          8         307.0         130    3504          12.0    70   
1    15.0          8         350.0         165    3693          11.5    70   
2    18.0          8         318.0         150    3436          11.0    70   
3    16.0          8         304.0         150    3433          12.0    70   
4    17.0          8         302.0         140    3449          10.5    70   
..    ...        ...           ...         ...     ...           ...   ...   
387  27.0          4         140.0          86    2790          15.6    82   
388  44.0          4          97.0          52    2130          24.6    82   
389  32.0          4         135.0          84    2295          11.6    82   
390  28.0          4         120.0          79    2625          18.6    82   
391  31.0          4         119.0          82    2720          19.4    82   

     origin                       name  
0         1  chevrolet

In [58]:
print (Auto_valid)

      mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
144  28.0          4          90.0          75    2125          14.5    74   
280  22.3          4         140.0          88    2890          17.3    79   
68   12.0          8         350.0         160    4456          13.5    72   
372  38.0          4         105.0          63    2125          14.7    82   
328  33.8          4          97.0          67    2145          18.0    80   
..    ...        ...           ...         ...     ...           ...   ...   
268  21.1          4         134.0          95    2515          14.8    78   
272  20.3          5         131.0         103    2830          15.9    78   
223  17.5          6         250.0         110    3520          16.4    77   
35   19.0          6         250.0          88    3302          15.5    71   
259  18.1          6         258.0         120    3410          15.1    78   

     origin                        name  
144       1          

In [59]:
print (Auto_train)

      mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
249  20.2          8         302.0         139    3570          12.8    78   
24   21.0          6         199.0          90    2648          15.0    70   
261  17.7          6         231.0         165    3445          13.4    78   
44   18.0          6         258.0         110    2962          13.5    71   
225  19.0          6         225.0         100    3630          17.7    77   
..    ...        ...           ...         ...     ...           ...   ...   
323  44.3          4          90.0          48    2085          21.7    80   
192  22.5          6         232.0          90    3085          17.6    76   
117  24.0          4         116.0          75    2158          15.5    73   
47   18.0          6         250.0          88    3139          14.5    71   
172  18.0          6         171.0          97    2984          14.5    75   

     origin                             name  
249       1     

Now we can fit a linear regression using only the observations corresponding to the training set `Auto_train`.

In [60]:
hp_mm = MS(['horsepower'])

X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']

model = sm.OLS(y_train, X_train)

results = model.fit()

summarize (results)


,coef,std err,t,P>|t|
intercept,39.9055,1.009,39.537,0.0
horsepower,-0.1563,0.009,-17.333,0.0


We now use the `predict()` method of results evaluated on the model matrix for this model created using the validation data set. We also calculate the validation MSE of our model.

In [61]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']

valid_pred = results.predict(X_valid)

# y_valid => real results used to validate the model
# valid_pred => prediction generated by the model, using the validation set
# the mean^2 of the (real results - predictied results) shows how close the prediction is from the reality

print ("MSE (Mean Squareed Error):" , np.mean((y_valid - valid_pred)**2) )

# the result is 23.61 ... this is the MSE
#    MSE measures the average squared error between actual and predicted values.
#    MSE (Mean Squareed Error) is how far the prediction is from the reality. 
#    The perfect model will have MSE = 0
#    Since there are no standard, you have to generate models and compare them using MSE.
#


MSE (Mean Squareed Error): 23.616617069669882


So our estimated test MSE for the linear regression is 23.62.

**What would happen if we used a different set of observations for our training set? Repeat the above
process using a different seed (i.e. set.seed(2)) and report the differences.**

## Leave-One-Out Cross-Validation

The simplest way to cross-validate in Python is to use `sklearn`. The `ISLP` package provides a wrapper (`sklearn_sm()`), that allows us to easily use the cross-validation tools of `sklearn` with models fit by `statsmodels`.


- LOOCV
- Advantage:
    - it is useful with small data sets
    - outliers/data-noise have low impact using this model
    - create a model, test it with 1 record then repeat the process 'N' number of times 
- Drawback
    - with large data sets this method is very time consuming and inefficinet to generate

In [64]:
print (Auto.shape)

print (Auto.shape[0])

(392, 9)
392


In [62]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))

X = Auto.drop(columns=['mpg'])
Y = Auto['mpg']

cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0]) #K-fold where k=n --> LOOCV


# score of all executions of the model 
print (cv_results['test_score'])

cv_err = np.mean(cv_results['test_score'])
print ("MSE (Mean Squareed Error): ",cv_err)



[2.02001002e+00 1.25092412e+00 3.06805164e+00 6.79901984e-02
 7.08255629e-01 4.13566745e+01 8.13755358e+01 6.71494767e+01
 9.70498847e+01 2.63430368e+01 3.67428697e+00 4.70741691e-01
 1.60507789e+00 9.70498847e+01 8.89557151e-01 8.69418110e+00
 4.41228965e+01 3.06562225e+01 9.16549732e-01 4.53185378e+01
 1.45705281e+00 3.00983554e+00 3.54617605e-03 1.52964088e+01
 2.25022208e+01 1.67905446e+01 2.76735351e+00 1.85354648e+01
 2.29957474e-01 9.16549732e-01 5.18379999e+00 3.54617605e-03
 2.66745510e+01 5.44791120e+01 5.14078324e+01 4.99405256e+01
 3.80360006e+01 1.19884585e-02 2.91033003e+00 3.23104363e+00
 5.16691118e+00 2.32487332e-01 1.06678908e-02 4.82615264e-01
 2.10211114e+01 4.35585197e+01 2.66745510e+01 6.51231162e+01
 1.13690418e+01 5.18379999e+00 1.25085727e+00 4.27873211e+00
 1.77161819e+00 3.58044876e+01 1.21519855e+01 8.41044041e+00
 8.89557151e-01 5.36657259e+00 7.17595842e+01 3.30230842e+01
 2.89239650e+01 8.09034686e-01 2.91033003e+00 1.60507789e+00
 3.23104363e+00 5.556481

The arguments to `cross_validate()` are: an object with the appropriate `fit()`, `predict()`, and `score()` methods, an array of features `X` and a response `Y`. We also included an additional argument `cv` to `cross_validate()`; specifying an integer 
$K$ results in $K$-fold cross-validation.

We can repeat this procedure for increasingly complex polynomial fits. To automate the process, we again use a for loop which iteratively fits polynomial regressions of degree 1 to 5, computes the associated cross-validation error, and stores it in the $i$th element of the vector `cv_error`.

**Write a for loop that fits the Auto data using polynomials up to degree 5. Compute the LOOCV
test error rate for each of them. Which degree polynomial fits the data the best according to
the test error rate and the bias-variance trade-off?**

Tip: use `np.power.outer` function to construct `X`

In [77]:
## Your codes
# mpg  cylinders  displacement  horsepower  weight  acceleration  year
cv_error = np.zeros (5)

H = np.array (Auto['horsepower'])
M = sklearn_sm(sm.OLS)
Y = Auto['mpg']

# i = 0,1,2,3,4
# d = 1,2,3,4,5
for i,d in enumerate (range (1,6)):
    
    X = np.power.outer (H , np.arange(d+1))
    
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])
    
    cv_error[i] = np.mean (M_CV['test_score'])

cv_error

# array([24.23151352, 19.24821312, 19.33498406, 19.4244303 , 19.03322606])
# d = 1 ==> 24.23151352
# d = 2 ==> 19.24821312
# d = 3 ==> 19.33498406
# d = 4 ==> 19.4244303
# d = 5 ==> 19.03322606
#
# the best model is d=5 because it has the lowest score.
# ... but ... the best choice is d=2 ... it is the second-best choice, but the model is much simpler.
# 

array([24.23151352, 19.24821312, 19.33498406, 19.4244303 , 19.03322606])

*These exercises were adapted from :* James, Gareth, et al. An Introduction to Statistical Learning: with Applications in Python, Springer, 2023.

# Validation Set Appoach #

- randomly divide the data set into : training set and validation set
- drawbacks:
    - this is not useful for small data sets
    - when randomly selecting the sets for TRAINING and VAIDATION it can produce very different models (speccialy in imbalanced data sets)

- 